In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Dropout, Conv2DTranspose, concatenate, GlobalAveragePooling2D, Dense
from tensorflow.keras.models import Model

# Define constants
NUM_CLASSES = 4  # Number of classes (correct, incorrect, partially correct, none)
IMG_WIDTH, IMG_HEIGHT = 224, 224  # Input image dimensions for U-Net
BATCH_SIZE = 32
EPOCHS = 15

# Build the U-Net model
inputs = Input(shape=(IMG_WIDTH, IMG_HEIGHT, 3))

# Encoder
conv1 = Conv2D(64, 3, activation='relu', padding='same')(inputs)
conv1 = Conv2D(64, 3, activation='relu', padding='same')(conv1)
pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)

conv2 = Conv2D(128, 3, activation='relu', padding='same')(pool1)
conv2 = Conv2D(128, 3, activation='relu', padding='same')(conv2)
pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)

conv3 = Conv2D(256, 3, activation='relu', padding='same')(pool2)
conv3 = Conv2D(256, 3, activation='relu', padding='same')(conv3)
pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)

conv4 = Conv2D(512, 3, activation='relu', padding='same')(pool3)
conv4 = Conv2D(512, 3, activation='relu', padding='same')(conv4)
drop4 = Dropout(0.5)(conv4)
pool4 = MaxPooling2D(pool_size=(2, 2))(drop4)

# Bottleneck
conv5 = Conv2D(1024, 3, activation='relu', padding='same')(pool4)
conv5 = Conv2D(1024, 3, activation='relu', padding='same')(conv5)
drop5 = Dropout(0.5)(conv5)

# Decoder
up6 = Conv2DTranspose(512, 2, strides=(2, 2), padding='same')(drop5)
concat6 = concatenate([up6, drop4])
conv6 = Conv2D(512, 3, activation='relu', padding='same')(concat6)
conv6 = Conv2D(512, 3, activation='relu', padding='same')(conv6)

up7 = Conv2DTranspose(256, 2, strides=(2, 2), padding='same')(conv6)
concat7 = concatenate([up7, conv3])
conv7 = Conv2D(256, 3, activation='relu', padding='same')(concat7)
conv7 = Conv2D(256, 3, activation='relu', padding='same')(conv7)

up8 = Conv2DTranspose(128, 2, strides=(2, 2), padding='same')(conv7)
concat8 = concatenate([up8, conv2])
conv8 = Conv2D(128, 3, activation='relu', padding='same')(concat8)
conv8 = Conv2D(128, 3, activation='relu', padding='same')(conv8)

up9 = Conv2DTranspose(64, 2, strides=(2, 2), padding='same')(conv8)
concat9 = concatenate([up9, conv1])
conv9 = Conv2D(64, 3, activation='relu', padding='same')(concat9)
conv9 = Conv2D(64, 3, activation='relu', padding='same')(conv9)

# Global Average Pooling
gap = GlobalAveragePooling2D()(conv9)

# Output
output = Dense(NUM_CLASSES, activation='softmax')(gap)

model = Model(inputs=inputs, outputs=output)

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Create data generators for training and validation
TRAINING_DIR = r"C:/Users/Lenovo/Desktop/koi bhi/train/"
train_datagen = ImageDataGenerator(
    rescale=1.0 / 255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

train_generator = train_datagen.flow_from_directory(
    TRAINING_DIR,
    target_size=(IMG_WIDTH, IMG_HEIGHT),
    batch_size=BATCH_SIZE,
    class_mode='categorical'
)

VALIDATION_DIR = r"C:/Users/Lenovo/Desktop/koi bhi/test/"
validation_datagen = ImageDataGenerator(rescale=1.0 / 255)

validation_generator = validation_datagen.flow_from_directory(
    VALIDATION_DIR,
    target_size=(IMG_WIDTH, IMG_HEIGHT),
    batch_size=BATCH_SIZE,
    class_mode='categorical'
)

# Train the model
model.fit(
    train_generator,
    steps_per_epoch=train_generator.n // BATCH_SIZE,
    epochs=EPOCHS,
    validation_data=validation_generator,
    validation_steps=validation_generator.n // BATCH_SIZE
)

# Save the model for later use
model.save('pose_classification_model_new_added_layer_U-Net.h5')


Found 6053 images belonging to 4 classes.
Found 1030 images belonging to 4 classes.
Epoch 1/15
  1/189 [..............................] - ETA: 3:54:44 - loss: 1.3864 - accuracy: 0.2812